In [2]:
import requests
import os
import time

host = 'http://127.0.0.1:8001'

session = requests.Session()


### Login

In [3]:
def login(username, password):
    url = host + '/api/login'
    r = session.post(url, json={'username': username, 'password': password})
    print(r.json())
    # print(r.headers)
    print(r.cookies)
    print(str(r))

In [21]:
login('YOUR_USERNAME', 'YOUR_PASSWORD')


{'id': 2, 'username': 'yxy123456', 'fullName': 'yxy123456', 'email': 'yxy123456@example.com', 'admin': False, 'loginAttempt': 0, 'projectLimited': 100, 'groupLimited': 10, 'createdTime': '2023-03-29T09:45:25.497+08:00', 'lastLoginTime': '2023-04-04T15:56:32.395+08:00', 'finishedUserGuide': True, 'locked': False, 'shouldChangePassword': False}
<RequestsCookieJar[<Cookie JSESSIONID=1F510C6C368F610006FD8C9C6DF6AB61 for 127.0.0.1/>]>
<Response [200]>


### Upload JSON

In [4]:
### get project id,need project id to upload json
def get_project_info(user_owner=None, group_owner=None, project_name=None):
    if user_owner is None and group_owner is None:
        raise Exception
    url = "%s/api/project" % (host)
    current_page = 1
    params = {
        'keyword': project_name,
        'page': 1,
        'pageSize': 100
    }
    response = session.get(url, params=params)
    content = response.json()
    if 200 <= response.status_code < 299:
        # print(content)
        # print(str(content['records']))
        for project in content['records']:
            # print(project['name'])
            # print(project_name)
            if project_name == project['name']:
                # print(project['name'])
                # owner = project['userOwner']
                if project['userOwner'] is not None:
                    owner = project['userOwner']
                    if user_owner == owner['username']:
                        return project

    else:
        print(str(content))

    return None

In [22]:
result_project_info = get_project_info(user_owner='yxy123456', group_owner=None, project_name='PUW Sample')

In [23]:
result_project_info

{'id': 17,
 'name': 'PUW Sample',
 'description': 'PUW Sample',
 'userOwnerId': 2,
 'groupOwnerId': None,
 'businessScenario': 30,
 'principleGeneric': True,
 'principleFairness': True,
 'principleEA': True,
 'principleTransparency': True,
 'currentModelArtifactVid': None,
 'currentQuestionnaireVid': None,
 'creatorUserId': 2,
 'createdTime': '2023-04-04T10:07:20.183+08:00',
 'lastEditedTime': '2023-04-04T10:07:20.183+08:00',
 'archived': False,
 'assessmentProgress': {'completed': 33, 'count': 50},
 'userOwner': {'id': 2,
  'username': 'yxy123456',
  'fullName': 'yxy123456',
  'email': 'yxy123456@example.com'},
 'groupOwner': None}

In [24]:
result_project_info['id']

17

In [26]:
def upload_json(project_id, json_path):
    # org.veritas.assessment.biz.controller.ProjectController.uploadJson

    # /{projectId}/modelArtifact
    # file content as http body
    url = "%s/api/project/%d/modelArtifact" % (host, project_id)
    files = {'file': ('puw.json', open(json_path, 'rb'), 'application/json')}
    response = session.post(url, files = files)
    print(response.status_code)
    print(response.json())

In [27]:
os.listdir()

['.DS_Store',
 'API function.ipynb',
 '.ipynb_checkpoints',
 'report_1.0.2.pdf',
 'report_1.0.3.pdf',
 'puw.json',
 'report_1.0.1.pdf',
 'report_1.0.0.pdf',
 'report_1.0.4.pdf']

In [28]:
upload_json(result_project_info['id'], "puw.json")


200
{'versionId': 23, 'projectId': 17, 'uploadUserId': 2, 'uploadTime': '2023-04-04T15:58:56.232+08:00', 'filename': 'puw.json', 'jsonContentSha256': '56bb1a71370c1c7f3ef35cb48d1d27e06c9eb9ffd261e9fd54182ea77153dd4f'}


### Export Report

In [29]:
def export(project_id, version, message):
    # org.veritas.assessment.biz.controller.ProjectReportController.exportPdf
    # /project/{projectId}/report/export
    # body:
    #   version
    #   message
    url = "%s/api/project/%d/report/export" % (host, project_id)
    data = {'version': version, 'message': message}
    # response = session.post(url, json = json.dumps(data))
    response = session.post(url, json = data)
    print("export")
    print(response.status_code)
    print(response.json())
    if 200 <= response.status_code < 300:
        return response.json()
    

In [30]:
version = "2.0.2"
message = 'v2.0.2 YOUR MESSAGE'
result_export_report = export(result_project_info['id'], version, message)


export
200
{'versionId': 16, 'projectId': 17, 'versionIdOfProject': 6, 'creatorUserId': 2, 'creator': {'id': 2, 'username': 'yxy123456', 'fullName': 'yxy123456', 'email': 'yxy123456@example.com'}, 'createdTime': '2023-04-04T16:00:47.222+08:00', 'tag': None, 'message': 'v2.0.2 YOUR MESSAGE', 'version': '2.0.2', 'modelArtifactVid': 23, 'questionnaireVid': '22547126353395712'}


In [31]:
result_export_report 

{'versionId': 16,
 'projectId': 17,
 'versionIdOfProject': 6,
 'creatorUserId': 2,
 'creator': {'id': 2,
  'username': 'yxy123456',
  'fullName': 'yxy123456',
  'email': 'yxy123456@example.com'},
 'createdTime': '2023-04-04T16:00:47.222+08:00',
 'tag': None,
 'message': 'v2.0.2 YOUR MESSAGE',
 'version': '2.0.2',
 'modelArtifactVid': 23,
 'questionnaireVid': '22547126353395712'}

### Download Report

In [32]:
def download_pdf(project_id, versionIdOfProject, filename):
    # org.veritas.assessment.biz.controller.ProjectVersionController.exportHistoryReport
    url = "%s/api/project/%d/history/%d/report" % (host, project_id, versionIdOfProject)

    response = session.get(url)
    print("download pdf")
    if 200 <= response.status_code < 300:
        content = response.content
        pdf = open(filename, 'wb')
        pdf.write(content)
    else:
        print(response.json())

In [33]:
filename = "report_%s.pdf" % result_export_report['version']
download_pdf(result_project_info['id'], result_export_report['versionIdOfProject'], filename)



download pdf


In [34]:
os.listdir()

['.DS_Store',
 'API function.ipynb',
 'report_2.0.2.pdf',
 '.ipynb_checkpoints',
 'report_1.0.2.pdf',
 'report_1.0.3.pdf',
 'puw.json',
 'report_1.0.1.pdf',
 'report_1.0.0.pdf',
 'report_1.0.4.pdf']